# Gaussian priors in infinite dimensions

In this notebook we show how to construct PDE-based priors that lead to well-posed Bayesian inverse problems in infinite dimesions.
Specifically, we will consider a Gaussian prior $\mu_{\rm prior} \sim \mathcal{N}( m_{\rm prior}, \mathcal{C}_{\rm prior} )$, where
the covariance operator $\mathcal{C}_{\rm prior}$ is defined as the inverse of an elliptic differential operator, i.e.

$$ \mathcal{C}_{\rm prior} = \left( \delta I - \gamma \Delta \right)^{-\alpha}, $$

equipped with homogeneous Neumann, Dirichlet or Robin boundary conditions, and $m_{\rm prior} \in H^{\frac{\alpha}{2}}(\Omega)$, where $\Omega \subset \mathbb{R}^d$.

The parameter $\alpha > \frac{d}{2}$ controls the smoothness of the random field and ensures that $\mathcal{C}_{\rm prior}$ is a trace class operator (i.e., the infinite sum of the eigenvalues of  $\mathcal{C}_{\rm prior}$ is finite). 
The fact that $\mathcal{C}_{\rm prior}$ is trace class is extremely important as it guaratees that the pointwise variance of the samples is finite. (Recall that for a Gaussian random field 
$ E [\int_{\Omega}(m - m_{\rm prior})^2\,dx = \operatorname{trace}(\mathcal{C}_{\rm prior})]$).

The parameters $\delta>0$, $\gamma>0$ can be constant in $\Omega$ (in this case the prior is called stationary) or spatially varing.

It can be shown that, in the case of the BiLaplacian prior ($\alpha=2$) in $d$-spatial dimensions, the relationship between the PDE coefficients $\gamma$ and $\delta$ and the desired marginal variance $\sigma^2$ and correlation length $\rho$ is

$$ \gamma = \frac{1}{s}, \quad \delta = \frac{\kappa^2}{s}, $$
where
$$ \nu = 2. - \frac{d}{2}, \quad \kappa = \frac{\sqrt{8\nu}}{\rho}, \text{ and } s = \sigma\kappa^\nu\sqrt{\frac{(4\pi)^{d/2}}{\Gamma(\nu)}}.$$


The above formulae are implemented in the `hIPPYlib` function `BiLaplacianComputeCoefficients`, which calculates the coefficients `gamma` and `delta` of the BiLaplacian prior ($\alpha=2$) given the desired marginal variance $\sigma^2$ and correlation length $\rho$.

## Import dependencies

In [ ]:
import dolfin as dl
import numpy as np
import math

from hippylib import *

import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.patches as mpatches
# from pylab import *
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r'\usepackage{amsfonts}')
%matplotlib inline

## Plot the marginal variance and correlation structure at a given point

In [ ]:
def correlationStructure(prior, center):
    rhs = dl.Vector()
    prior.init_vector(rhs, 0)
    
    corrStruct = dl.Vector()
    prior.init_vector(corrStruct, 0)
    
    ps = dl.PointSource(prior.Vh, center, 1.)
    ps.apply(rhs)
    
    prior.Rsolver.solve(corrStruct, rhs)
    
    return vector2Function(corrStruct, prior.Vh)

def makeCorrelationPlots(mesh, sigma2, rho, coords,plot_prefix = '',anisotropic=False,\
                  degree = 2, tick_size = 18, title_size = 32, use_robin = True,\
                         theta0=2.0, theta1 =0.5, alpha = math.pi/4, cmap='viridis'):
    """
    Show the plot of the marginal variance and correlation structure
    - mesh: The finite element mesh
    - sigma2: The prescribed marginal variance
    - rho: The prescribed correlation lenghth
    - pt: Point at which to show the correlation structure
    """
    ndim = mesh.geometric_dimension()
    Vh = dl.FunctionSpace(mesh, "CG", 1)

    gamma, delta = BiLaplacianComputeCoefficients(sigma2, rho, ndim)

    
    if degree == 1:
        prior = LaplacianPrior(Vh, gamma, delta)
    elif degree == 2:
        if anisotropic:
            anis_diff = dl.CompiledExpression(ExpressionModule.AnisTensor2D(), degree = 1)
            anis_diff.set(theta0, theta1, alpha)
        else:
            anis_diff = None
        prior = BiLaplacianPrior(Vh, gamma, delta,  anis_diff, robin_bc=use_robin)
        prior.Asolver = dl.PETScLUSolver(dl.as_backend_type(prior.A)) #Replace iterative with direct solver
    else:
        raise


    ## pointwise variance
    r = int(4*np.sqrt(Vh.dim()))
    pointwise_variance = vector2Function(prior.pointwise_variance(method="Randomized", r=r), Vh)

    # construct pt
    pt = dl.Point(coords)
    
    correlation_struc  = correlationStructure(prior, pt)

    print("Prescribed marginal variance: ", sigma2)
    print("Actual marginal variance:", correlation_struc.vector().norm("linf"))
    
    ## Plot variance and correlation structure
    if ndim == 1:
        fig, axes = plt.subplots(1, 2, figsize=[15,6])
        for ax in axes:
            ax.tick_params(axis='both', which='major', labelsize=tick_size)
            ax.tick_params(axis='both', which='minor', labelsize=tick_size)

        plt.sca(axes[0])
        dl.plot(pointwise_variance)
        plt.title("Marginal variance",fontsize=title_size)
        plt.sca(axes[1])
        dl.plot(correlation_struc)
        # Adding some plot annotations
        ymax = np.max(correlation_struc.vector().get_local())
        
        plt.vlines(x=[coords], ymin=[0], ymax=[0.98*ymax],\
                           colors='black', ls='--', lw=2)
        
        arr = mpatches.FancyArrowPatch((coords, 0.1*ymax), (coords+rho, 0.1*ymax),
                               arrowstyle='<->,head_width=.15', mutation_scale=20)
        ax.add_patch(arr)
        bbox=dict(fc="white", ec="none")
        ax.annotate('${\\rho}='+str(rho)+'$', (.5, .5), xycoords=arr, ha='center', va='center',\
                                    fontsize=15,bbox=bbox)

        plt.title("Correlation",fontsize=title_size)
        fig.savefig(plot_prefix+'var_and_correlation.pdf')
    else:
        nb.multi1_plot([pointwise_variance,correlation_struc],['Marginal variance','Correlation'],\
                      fontsize = title_size,cmap=cmap)
        fig, ax = plt.gcf(), plt.gca()
        ims = ax.images 
        for im in ims:
            cb = im.colorbar
            cb = fig.colorbar()
            cb.ax.tick_params(labelsize=tick_size)
        fig.savefig(plot_prefix+'var_and_correlation.pdf')
    
    plt.show()
    
    
    
save_dir = 'figures/'
os.makedirs(save_dir,exist_ok = True)

### The 1D case

In [ ]:
mesh = dl.UnitIntervalMesh(100)
sigma2 = 4
rho    = 0.25
coords = 0.5

makeCorrelationPlots(mesh, sigma2, rho, coords,plot_prefix = save_dir+'1D05')

coords = 0.75

makeCorrelationPlots(mesh, sigma2, rho, coords,plot_prefix = save_dir+'1D05norobin',use_robin=False)

### The 2D case

In [ ]:
mesh = dl.UnitSquareMesh(64,64)
sigma2 = 4
rho    = 0.25
coords = (0.5,0.5)

makeCorrelationPlots(mesh, sigma2, rho, coords,plot_prefix = save_dir+'2D0505',cmap='jet')

coords = (0.75,0.75)
makeCorrelationPlots(mesh, sigma2, rho, coords,plot_prefix = save_dir+'2D075075',cmap='jet')

coords = (0.75,0.75)
makeCorrelationPlots(mesh, sigma2, rho, coords,plot_prefix = save_dir+'2D075075norobin',cmap='jet',use_robin = False)

## Generating Samples

In [ ]:
def makeSamplePlots(mesh, sigma2, rho, plot_prefix = '',anisotropic=False,\
                  tick_size = 18, title_size = 32, use_robin = True,\
                   degree = 2,theta0=2.0, theta1 =0.5, alpha = math.pi/4,\
                    with_correlation = False,same_colorbar = True, cmap = None):
    """
    Show the plot of the marginal variance and correlation structure
    - mesh: The finite element mesh
    - sigma2: The prescribed marginal variance
    - rho: The prescriced coerrelation lenghth
    - pt: Point at which to show the correlation structure
    """
    ndim = mesh.geometric_dimension()
    Vh = dl.FunctionSpace(mesh, "CG", 1)
    
    gamma, delta = BiLaplacianComputeCoefficients(sigma2, rho, ndim)
    
    if degree == 1:
        prior = LaplacianPrior(Vh, gamma, delta)
    elif degree == 2:
        if anisotropic:
            anis_diff = dl.CompiledExpression(ExpressionModule.AnisTensor2D(), degree = 1)
            anis_diff.set(theta0, theta1, alpha)
        else:
            anis_diff = None
        prior = BiLaplacianPrior(Vh, gamma, delta,  anis_diff, robin_bc=use_robin)
        prior.Asolver = dl.PETScLUSolver(dl.as_backend_type(prior.A)) #Replace iterative with direct solver
    else:
        raise

    

    
    def sample(prior):
        noise = dl.Vector()
        prior.init_vector(noise,"noise")
        parRandom.normal(1., noise)
        mtrue = dl.Vector()
        prior.init_vector(mtrue, 0)
        prior.sample(noise,mtrue)
        return mtrue
    
    plot_items = [ vector2Function(sample(prior),Vh) for i in range(2)]
    labels = ['Sample 1','Sample 2']
    
    if with_correlation:
        # construct pt
        coords = tuple(ndim*[0.5])
        pt = dl.Point(coords)
        correlation_struc  = correlationStructure(prior, pt)
        plot_items = [correlation_struc] + plot_items
        labels = ['Correlation']+ labels
    
    nb.multi1_plot(plot_items,labels,same_colorbar = same_colorbar,\
                  fontsize = title_size, cmap = cmap)
    fig, ax = plt.gcf(), plt.gca()
    ims = ax.images 
    for im in ims:
        cb = im.colorbar
        cb = fig.colorbar()
        cb.ax.tick_params(labelsize=tick_size)
    fig.savefig(plot_prefix+'samples.pdf')
    
    plt.show()

### Isotropic samples

In [ ]:
mesh = dl.UnitSquareMesh(64,64)
sigma2 = 4
rho    = 0.25
makeSamplePlots(mesh,sigma2,rho,title_size =25,plot_prefix=save_dir+'isotropic_64')

### Anisotropic samples with $\alpha = \frac{\pi}{4}$

In [ ]:
mesh = dl.UnitSquareMesh(64,64)
sigma2 = 4
rho    = 0.25
kwargs = {'theta0':2., 'theta1':0.5, 'alpha':math.pi/4,'anisotropic':True}
makeSamplePlots(mesh,sigma2,rho,title_size =25,plot_prefix=save_dir+'anisotropic_64pi4',**kwargs)

coords = (0.5,0.5)

makeCorrelationPlots(mesh, sigma2, rho, coords,plot_prefix = save_dir+'2D0505anispi4',**kwargs,cmap='jet')

kwargs['cmap'] = ['jet','viridis','viridis']
kwargs['same_colorbar'] = False
makeSamplePlots(mesh,sigma2,rho,title_size =25,plot_prefix=save_dir+'threeplotanisotropic_64pi4',**kwargs,with_correlation = True)


### Anisotropic samples with $\alpha = -\frac{\pi}{4}$


In [ ]:
mesh = dl.UnitSquareMesh(64,64)
sigma2 = 4
rho    = 0.25
kwargs = {'theta0':2., 'theta1':0.5, 'alpha':-math.pi/4,'anisotropic':True}
makeSamplePlots(mesh,sigma2,rho,title_size =25,plot_prefix=save_dir+'anisotropic_64minuspi4',**kwargs)

coords = (0.5,0.5)
makeCorrelationPlots(mesh, sigma2, rho, coords,plot_prefix = save_dir+'2D0505anisminuspi4',**kwargs,cmap='jet')

kwargs['cmap'] = ['jet','viridis','viridis']
kwargs['same_colorbar'] = False
makeSamplePlots(mesh,sigma2,rho,title_size =25,plot_prefix=save_dir+'threeplotanisotropic_64minuspi4',**kwargs,with_correlation = True)

## Mesh Dependence and Independence

In [ ]:
def locallyRefinedMesh():
    mesh = dl.UnitSquareMesh(16,16)
    
    for i in range(4):
        cell_markers = dl.MeshFunction("bool", mesh,2)
        cell_markers.set_all(False)
        for cell in dl.cells(mesh):
            if cell.midpoint()[1] < .7 and cell.midpoint()[1] > .3 and cell.midpoint()[0] > .2 and cell.midpoint()[0] < .5:
                cell_markers[cell] = True
            
        mesh = dl.refine(mesh, cell_markers)
        
    return mesh

mesh1 = dl.UnitSquareMesh(16,16)
mesh2 = dl.UnitSquareMesh(64, 64)
mesh3 = locallyRefinedMesh()

nb.multi1_plot([mesh1, mesh2, mesh3], ["Coarse mesh", "Fine mesh", "Locally refined"])
fig, ax = plt.gcf(), plt.gca()
fig.savefig(save_dir+'meshes.pdf')


### Laplacian plots

In [ ]:
sigma2 = 4
rho    = 0.25
coords = (0.5,0.5)

meshes = {'coarse':mesh1,'fine':mesh2,'refined':mesh3}

for (name,mesh) in meshes.items():
    makeSamplePlots(mesh,sigma2,rho,title_size =25,plot_prefix=save_dir+name+'isotropic_64pi4')
    makeCorrelationPlots(mesh, sigma2, rho, coords,degree = 1,plot_prefix = save_dir+name+'2D0505laplace',**kwargs,cmap='jet')



### BiLaplacian Plots

In [ ]:
sigma2 = 4
rho    = 0.25
coords = (0.5,0.5)

meshes = {'coarse':mesh1,'fine':mesh2,'refined':mesh3}

for (name,mesh) in meshes.items():
    makeSamplePlots(mesh,sigma2,rho,title_size =25,plot_prefix=save_dir+name+'isotropic_64pi4')
    makeCorrelationPlots(mesh, sigma2, rho, coords,degree = 2, plot_prefix = save_dir+name+'2D0505bilaplace',**kwargs,cmap='jet')


Copyright &copy; 2016-2018, The University of Texas at Austin & University of California, Merced.<br>
Copyright (c) 2019-2022, The University of Texas at Austin, University of California--Merced, Washington University in St. Louis.<br>
Copyright (c) 2023-, The University of Texas at Austin, University of California--Merced.<br>
All Rights reserved.<br>
See file COPYRIGHT for details.

This file is part of the hIPPYlib library. For more information and source code
availability see https://hippylib.github.io.

hIPPYlib is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License (as published by the Free Software Foundation) version 2.0 dated June 1991.